# Modificaciones a "Generador del Dataset"

In [1]:
import re
from unicodedata import normalize

## Función `strip_accents`:

In [2]:
def strip_accents(string=None):
    # -> NFD y eliminar diacríticos
    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
               normalize( "NFD", string), 0, re.I)
    # -> NFC
    return normalize( 'NFC', s).upper()


Cambios:
- Conviene no utilizar `None` como argumento default ya que 1) es el único argumento de la función y 2) esta función no puede hacer nada útil si no tiene un argumento.
- Se puede evitar pasar `0` (que ya es el default) a `sub()` usando `flags=re.I`.
- Usar entrecomillado consistente (' o ").
- Cambios a indentación (max 79 columnas).
- Conviene no utilizar nombres demasiado similares a built-ins (`string` ~ `str`)
- La expresión regular es bastante compleja (tres paréntesis anidados), puede ser difícil de interpretar

In [3]:
def strip_accents(string):
    # -> NFD y eliminar diacríticos
    string = re.sub(
        r'([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+',
        r'\1',
        normalize('NFD', string),
        flags=re.I
    )

    # -> NFC
    return normalize('NFC', string).upper()

In [4]:
s = strip_accents('NoréstÉñÃ, N°º âgüìta')
s

'NORESTEÑA, N°º AGUITA'

## Función `split_streets`:

In [5]:
def split_streets(string):
    string = string.strip()
    string = re.sub(r'(?=[0-9])(?<=[A-Z])', r' ', string)
    string = re.sub(r'(?<=[0-9])(?=[A-Z])', r' ', string)
    streets = re.compile(r'([A-Z0-9\s°º,.]+)\s(E|E/|ENT|ENTRE)\s([A-Z0-9\s.]+)\sY\s([A-Z0-9\s°º.]+)')
    # Esta expresion regular no contempla el caso "calle1 ####, esquina Calle2"
    result = streets.search(string=string)
    if result:
        calle1 = result.groups()[0]
        calle2 = result.groups()[2]
        calle3 = result.groups()[3]
    else:
        streets = re.compile(r'([A-Z0-9\s°º.]+)\sY\s([A-Z0-9\s°º.]+)')
        result = streets.search(string=string)
        if result:
            calle1, calle2 = result.groups()
            calle3 = None
        else:
            calle1 = string
            calle2 = None
            calle3 = None
    return calle1, calle2, calle3

Cambios:
- Conviene no utilizar nombres demasiado similares a built-ins (`string` ~ `str`)
- La expresión regular `(?=[0-9])(?<=[A-Z])` es un poco confusa porque pone la lookahead assertion primero y luego una lookbehind.
  Se podría modificar para que 1) no use assertions 2) tenga el orden correcto.
- La expresión regular `'(?<=[0-9])(?=[A-Z])'` podría ser modificada para que no use assertions.
- Cambios a indentación (max 79 columnas).
- Utilizar inglés consistentemente (para código).
- Llamar a `compile` manualmente es innecesario ya que `re` mantiene un cache con las últimas ER utilizadas. Si igualmente queremos compilarla, es mejor hacerlo
  a nivel módulo (para asegurarse de que nunca se pierda su versión cacheada). El llamado a `compile` debería ser considerado como algo caro que no queremos hacer en cada
  llamada a la función.
- Se podría cambiar la clase `[A-Z0-9\s°º.]` por `[\w\s°º.]` y combinarla con `re.IGNORECASE` para no tener que utilizar `strip_accents`.
- Se podría cambiar la clase `[A-Z]` por `[^\d\W]` y combinarla con `re.IGNORECASE` para no tener que utilizar `strip_accents`.
- Se podría cambiar la clase `[0-9]` por `\d`.
- Se puede utilizar `Match.group()` con varios argumentos (índices) para retornar tuplas de resultados directamente.
- Cambios estructurales con early returns
- Normalzación de espacios

In [6]:
# Esta expresion regular no contempla el caso "calle1 ####, esquina Calle2"
STREET_PATTERN_3 = \
    r'([\w\s°º,.]+)\s(E|E/|ENT|ENTRE)\s([\w\s.]+)\sY\s([\w\s°º.]+)'

STREET_PATTERN_2 = r'([\w\s°º.]+)\sY\s([\w\s°º.]+)'

def split_streets(string):
    string = ' '.join(string.split())
    string = re.sub(r'(\d)([^\d\W])', r'\1 \2', string, flags=re.I)
    string = re.sub(r'([^\d\W])(\d)', r'\1 \2', string, flags=re.I)

    result = re.search(STREET_PATTERN_3, string, re.I)

    if result:
        return result.group(1, 3, 4)

    result = re.search(STREET_PATTERN_2, string, re.I)
    if result:
        street1, street2 = result.groups()
        return street1, street2, None

    return string, None, None

In [7]:
# Test de la función 'split_streets()'
string = [
    'BELGRANO Y ENTRE RIOS','RUTA 8 KM 52','F.ALCORTA Y PAMPA',
    'SAN JUAN DE DIOS 666, ENT BELGRANO Y ENTRE RIOS',
    'YATAY 541',
    'CALLE 41 12 E/ 1 Y 2',
    'Arribeños 5743',
    'Tucumán 302 entre Córdoba y Entre Ríos',
    'Arribeños33'
]

for s in string:
    print(split_streets(s))

('BELGRANO', 'ENTRE RIOS', None)
('RUTA 8 KM 52', None, None)
('F.ALCORTA', 'PAMPA', None)
('SAN JUAN DE DIOS 666,', 'BELGRANO', 'ENTRE RIOS')
('YATAY 541', None, None)
('CALLE 41 12', '1', '2')
('Arribeños 5743', None, None)
('Tucumán 302', 'Córdoba', 'Entre Ríos')
('Arribeños 33', None, None)


## Función `numeracion`:

In [8]:
def numeracion(string):
    numer = re.compile('([\s][N°ºKM]{,2})[\s]{0,}([0-9]+)')

    delete = ['CALLE','AVENIDA','AV.','AV','RUTA','NACIONAL','NAC.','NAC','PROVINCIAL','PROV.','PROV',
              'DIAGONAL','DIAG.','DIAG','BOULEVARD','BV.','BV'] # Esta parte es complicada.... No me gusta

    for word in delete:
        try:
            string = string.replace(word,'')
        except:
            pass
    try:
        string = string.strip()
        string = re.sub(r'(?=[0-9])(?<=[A-Z])', r' ', string)
        string = re.sub(r'(?<=[0-9])(?=[A-Z])', r' ', string)
        result = numer.search(string=string)
        return result.group().strip()
    except:
        return None

Cambios:
- Conviene no utilizar nombres demasiado similares a built-ins (`string` ~ `str`, `delete` ~ `del`).
- Usar inglés para el código.
- Misma observación sobre `compile`.
- La lista de strings podría ponerse afuera de la función para que no se cree nuevamente en cada llamado.
- Se puede reemplazar `[0-9]` con `\d`.
- Se puede reemplazar `[\s]` con `\s`.
- Se puede reemplazar `{0,}` con `*`.
- `[N°ºKM]{,2}` puede llegar a matchear `MM` o `°°`, reemplazar con `(n[°º]?|km)?`.
- Usar raw strings (r'') para evitar secuencias de escape inválidas
- La parte que separa letras y números es igual a la de `split_streets`, se podría sacar esa parte a otra función.
- El try/except con `str.replace()` no es necesario ya que la función no lanza excepciones.
  + `except: pass` no debe usarse porque ignora *todos* los errores posibles (raramente/casi nunca es lo que queremos)
- El segundo try/except es demasiado general (no especifica tipos de excepciones), y podría cambiarse con un `if`, ya que
  la única parte del código que puede fallar es el `.strip()` de `result.group()`.

In [9]:
# Version 1
def separate_alpha_nums(string):
    string = re.sub(r'(\d)([^\d\W])', r'\1 \2', string, flags=re.I)
    return re.sub(r'([^\d\W])(\d)', r'\1 \2', string, flags=re.I)

 # Esta parte es complicada.... No me gusta
STREET_KEYWORDS = ['CALLE', 'AVENIDA', 'AV.','AV', 'RUTA', 'NACIONAL', 'NAC.',
                   'NAC', 'PROVINCIAL', 'PROV.', 'PROV', 'DIAGONAL', 'DIAG.',
                   'DIAG', 'BOULEVARD', 'BV.', 'BV']

STREET_NUM_REGEXP = r'\s(n[°º]?|km)?\s*(\d+)'

def street_number(string):
    for word in STREET_KEYWORDS:
        string = string.replace(word,'')

    string = separate_alpha_nums(string).strip()
    result = re.search(STREET_NUM_REGEXP, string, flags=re.I)
    return result.group().strip() if result else None

In [10]:
# Test de la función 'street_number()'
string = ['JOSE HERNANDEZ N°4859 8D','JOSE HERNANDEZ N  4859 8D','JOSE HERNANDEZ N4859 8TOD','35 N 2710',
          '141 BIS 4680','3458','GURRUCHAGA','CARMEN Nº335 5°C', 'PERÓN2019','25 DE MAYO 243 7A','552 BIS','41',
          'RUTA 8 KM 52','AV. 25 DE MAYO 322','CALLE 543 59']#,'RUTA 36 KM 41 1/2']<= Con este no funciona...

for s in string:
    print(street_number(s))

N°4859
N  4859
N 4859
N 2710
4680
None
None
Nº 335
2019
243
None
None
KM 52
322
59


## Función `unidades`

In [11]:
def unidades(string):
    try:
        string = string.strip() 
        unid = re.compile('([N°ºKM]{,2})[\s]{0,}([0-9]+)')
        result = unid.search(string=string)
        ans = result.groups()
    except:
        ans = (None,None)
    return ans

Cambios:
- Misma observación con `compile` y nombre de variable `string`.
- También mismas observaciones con detalles de la expresión regular `([N°ºKM]{,2})[\s]{0,}([0-9]+)`.
- Código en inglés
- El try/except es demasiado abarcativo (no especfica tipo de excepción)
  + Igualmente se lo puede remover porque sabemos puntualmente dónde se pueden dar errores
- No es necesario que la función maneje casos donde `string == None` (ver precondiciones y postcondiciones)

In [12]:
def units(string):
    string = string.strip()
    result = re.search(r'(n[°º]?|km)?\s*(\d+)', string, re.I)
    return result.groups() if result else (None, None)

In [13]:
string = ['N°4859','N 4859','4680''Nº 335','KM 52']

for s in string:
    print(units(s))

('N°', '4859')
('N', '4859')
(None, '4680')
('KM', '52')


## Función `normalización`

In [14]:
def normalizacion(string):
    string = strip_accents(string)
    calle1,calle2,calle3 = split_streets(string)
    num1 = numeracion(calle1)
    if type(num1) is str:
        calle1 = calle1.split(num1)[0]
    num2 = numeracion(calle2)
    if type(num2) is str:
        calle2 = calle2.split(num2)[0]
    num3 = numeracion(calle3)
    if type(num3) is str:
        calle3 = calle3.split(num3)[0]
    num = [num1,num2,num3]
    for n in num:
        if n is None:
            num[num.index(n)] = ''
        else:
            pass
    num = ''.join(map(str, num))
    unid,num = unidades(num)
    return calle1,calle2,calle3,unid,num

Cambios:
- Salteo la parte de `strip_accents`
- Nombres en inglés
- Usar conversión implícita a `bool` para checkear si un string está presente (no `type()`)
  + Incluso si se quiere checkear el tipo de algo, es mejor usar `isinstance()`
  + Igualmente en muy pocos casos (y muy específicos) es necesario checkear tipos (ver 'Duck Typing')
- Hay un bloque de código que se repite 3 veces, es mejor trabajar con listas y usar `for`
- No es conveniente modificar un contenedor mientras se está iterando sobre el mismo
- Se pueden utilizar list comprehensions para extraer los valores no-`None` de la lista
- `else`: `pass` no tiene efectos
- Al devolver tantos valores, es conveniente utilizar un diccionario (`dict`) en vez de una tupla.

In [15]:
def normalize_address(string):
    streets = split_streets(string)
    street_names = []
    numbers = []

    for street in streets:
        number = street_number(street) if street else None

        if number:
            street_names.append(street.split(number)[0])
        else:
            street_names.append(street)

        numbers.append(number)

    number_and_unit = ''.join([str(n) for n in numbers if n])
    unit, number = units(number_and_unit)

    return {
        'street_names': street_names,
        'number': number,
        'unit': unit
    }

    

In [16]:
addresses = ['BELGRANO Y ENTRE RIOS','RUTA 8 KM 52','F.ALCORTA Y PAMPA',
             'SAN JUAN DE DIOS N°666, ENT BELGRANO Y ENTRE RIOS',
             'YATAY 541', 'CALLE 41 12 E/ 1 Y 2',
             'ENTRE RIOS 334 E/ TUCUMAN Y INDEPENDENCIA',
             'Tucumán N° 3443 entre Córdoba y Entre Ríos']

for address in addresses:
    print(normalize_address(address))

{'street_names': ['BELGRANO', 'ENTRE RIOS', None], 'number': None, 'unit': None}
{'street_names': ['RUTA 8 ', None, None], 'number': '52', 'unit': 'KM'}
{'street_names': ['F.ALCORTA', 'PAMPA', None], 'number': None, 'unit': None}
{'street_names': ['SAN JUAN DE DIOS ', 'BELGRANO', 'ENTRE RIOS'], 'number': '666', 'unit': 'N°'}
{'street_names': ['YATAY ', None, None], 'number': '541', 'unit': None}
{'street_names': ['CALLE 41 ', '1', '2'], 'number': '12', 'unit': None}
{'street_names': ['ENTRE RIOS ', 'TUCUMAN', 'INDEPENDENCIA'], 'number': '334', 'unit': None}
{'street_names': ['Tucumán ', 'Córdoba', 'Entre Ríos'], 'number': '3443', 'unit': 'N°'}
